### tools

In [5]:
def add(a: int, b: int) -> int:
    """Add two numbers.
    
    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of a and b.
    """
    return a + b

In [7]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()

True

In [8]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [9]:
llm = ChatGroq(
    model = "qwen/qwen3-32b")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000018707211340>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018706F95760>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
## bind tools with llm

llm_with_tools =  llm.bind_tools([add])


tool_call = llm_with_tools.invoke([HumanMessage(content= f"what is the sum of 2 and 3?", name = "User")])

In [13]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'a': 2, 'b': 3},
  'id': '9p34z18fv',
  'type': 'tool_call'}]

## using messages as state

In [21]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from typing import Annotated

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [22]:
## reducers with add_messages

initial_messages = [AIMessage(content= f"Hello! how can i help you?", name = "LLMModel")]
initial_messages.append(HumanMessage(content= f"Can you tell me a joke?", name = "User"))
# initial_messages.append(AIMessage(content= f"which programming languare you want to learn", name = "LLMModel"))
# initial_messages.append(HumanMessage(content= f"i want to learn python language", name = "User"))


initial_messages

[AIMessage(content='Hello! how can i help you?', additional_kwargs={}, response_metadata={}, name='LLMModel', tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Can you tell me a joke?', additional_kwargs={}, response_metadata={}, name='User')]

In [23]:
AI_messages=AIMessage(content= f"which programming languare you want to learn", name = "LLMModel")   
AI_messages

AIMessage(content='which programming languare you want to learn', additional_kwargs={}, response_metadata={}, name='LLMModel', tool_calls=[], invalid_tool_calls=[])

In [25]:
add_messages(initial_messages, AI_messages)

[AIMessage(content='Hello! how can i help you?', additional_kwargs={}, response_metadata={}, name='LLMModel', id='769aee92-7340-4b4b-b68c-dc19e8e2bfa3', tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Can you tell me a joke?', additional_kwargs={}, response_metadata={}, name='User', id='ab58b804-bd1d-440b-83f9-cb25987199de'),
 AIMessage(content='which programming languare you want to learn', additional_kwargs={}, response_metadata={}, name='LLMModel', id='91d3b2ee-a625-4cc6-89a4-e302f648b5e1', tool_calls=[], invalid_tool_calls=[])]

In [38]:
## chatbot node functionality with tools

def llm_tool(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [39]:
from IPython.display import display, Markdown, Image
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolExecutor

# Tool executor for handling function calls
tool_executor = ToolExecutor([add])

def execute_tools(state: State):
    """Execute the tools called by the LLM"""
    messages = state["messages"]
    last_message = messages[-1]
    
    tool_results = []
    for tool_call in last_message.tool_calls:
        tool_result = tool_executor.invoke({
            "tool_name": tool_call["name"],
            "tool_input": tool_call["args"]
        })
        tool_results.append(tool_result)
    
    return {"messages": tool_results}

def router(state: State):
    """Route based on whether tools were called"""
    messages = state["messages"]
    last_message = messages[-1]
    
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

builder = StateGraph(State)

builder.add_node("llm_tool", llm_tool)
builder.add_node("tools", execute_tools)

builder.add_edge(START, "llm_tool")
builder.add_conditional_edges("llm_tool", router, {"tools": "tools", END: END})
builder.add_edge("tools", "llm_tool")

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

ImportError: cannot import name 'ToolExecutor' from 'langgraph.prebuilt' (c:\Users\User\anaconda3\envs\LangGraphv1\Lib\site-packages\langgraph\prebuilt\__init__.py)

In [ ]:
from langchain_core.messages import HumanMessage

messages = graph.invoke({
    "messages": [HumanMessage(content="what is 2 + 3?")]
})

print("Final messages:")
for message in messages["messages"]:
    message.pretty_print()

================================ Human Message =================================

what is 2 + 2
================================== Ai Message ==================================
Tool Calls:
  add (bf580abs4)
 Call ID: bf580abs4
  Args:
    a: 2
    b: 2
